In [1]:
import requests
import pandas as pd

API_KEY = 'your_tmdb_api_key'
BASE_URL = 'https://api.themoviedb.org/3'

def get_movie_details(movie_id):
    url = f'{BASE_URL}/movie/{movie_id}?api_key={API_KEY}&language=en-US'
    response = requests.get(url)
    return response.json()

def get_cast_and_crew(movie_id):
    url = f'{BASE_URL}/movie/{movie_id}/credits?api_key={API_KEY}&language=en-US'
    response = requests.get(url)
    return response.json()

def main():
    movies = []
    
    # TMDb does not provide a direct way to filter Bollywood movies, so we need to search for popular movies and filter them manually
    for page in range(1, 51):  # Adjust the range to get 1000 movies (20 movies per page, 50 pages)
        url = f'{BASE_URL}/movie/popular?api_key={API_KEY}&language=en-US&page={page}'
        response = requests.get(url)
        data = response.json()
        
        for movie in data['results']:
            movie_id = movie['id']
            details = get_movie_details(movie_id)
            
            # Filter for Bollywood movies (if available in data)
            if details['original_language'] == 'hi':  # Assuming 'hi' for Hindi
                cast_crew = get_cast_and_crew(movie_id)
                
                movie_data = {
                    'movie_id': movie_id,
                    'title': details.get('title', ''),
                    'overview': details.get('overview', ''),
                    'genres': [genre['name'] for genre in details.get('genres', [])],
                    'keywords': [keyword['name'] for keyword in details.get('keywords', {}).get('keywords', [])],
                    'cast': [cast['name'] for cast in cast_crew.get('cast', [])],
                    'crew': [crew['name'] for crew in cast_crew.get('crew', [])]
                }
                movies.append(movie_data)
                
                if len(movies) >= 1000:
                    break
        if len(movies) >= 1000:
            break
    
    df = pd.DataFrame(movies)
    df.to_csv('bollywood_movies.csv', index=False)
    print("Data saved to bollywood_movies.csv")

if __name__ == "__main__":
    main()


KeyError: 'results'